In [2]:
library(xlsx)
library(readxl)
library(stringr)
library("dplyr")
library( readxl )

In [3]:
path = getwd()
path = substr(path, 1, nchar(path)-9)
setwd(paste0(path, "data"))
getwd()

[1] "/Users/chen.liang/Desktop/变量/splc_analysis/data"

# Read in Data

In [4]:
get_semiconductor = function(){
    path = paste0(getwd(), "/Semi-Conductor/da_equipment.xlsx")
    data_eq = data.frame()
    for(i in 1:length(excel_sheets(path))){
        input = read.xlsx(path, i)
        data_eq = rbind(data_eq, input)
    }

    path = paste0(getwd(), "/Semi-Conductor/da_materials.xlsx")
    data_ma = data.frame()
    for(i in 1:length(excel_sheets(path))){
        input = read.xlsx(path, i)
        data_ma = rbind(data_ma, input)
    }

    path = paste0(getwd(), "/Semi-Conductor/da_storage.xlsx")
    data_st = data.frame()
    for(i in 1:length(excel_sheets(path))){
        input = read.xlsx(path, i)
        data_st = rbind(data_st, input)
    }

    path = paste0(getwd(), "/Semi-Conductor/da_yuanqi.xlsx")
    data_yq = data.frame()
    for(i in 1:length(excel_sheets(path))){
        input = read.xlsx(path, i)
        data_yq = rbind(data_yq, input)
    }

    data_eq$type = "eq"
    data_ma$type = "ma"
    data_st$type = "st"
    data_yq$type = "yq"

    data = rbind(data_eq, data_ma, data_st, data_yq)
    return(data)
}

get_phones = function(){
    path = paste0(path, "/CellPhone/AllPhones1009.xlsx")
    samsung = read.xlsx(path, "samsung")
    xiaomi = read.xlsx(path, "xiaomi")
    apple = read.xlsx(path, "AAPL")
    huawei = read.xlsx(path, "KMCACZ CH Equity")
    data = rbind(samsung,xiaomi, apple, huawei)
    return(data)
}

In [6]:
data = get_semiconductor()
data[0:1,]
nrow(data)

com,com_N,sup,sup_N,cus,cus_N,rel_V_fr_S,rel_V_to_C,sup_country,sup_gics,cus_country,cus_gics,type
AMAT US Equity,Applied Materials Inc,FLEX US Equity,Flex Ltd,2330 TT Equity,Taiwan Semiconductor Manufacturing Co Ltd,74.485176,463.06,US,Technology Hardware & Equipmen,TW,Semiconductors & Semiconductor,eq


[1] 12765

### [Optional] Generate Node Request for Nodes not in "Nodes_Data.csv"

In [8]:
data = data %>% mutate_at(vars(com, sup, cus), as.character)
all_companies_ticker = unique(c(data$com, data$sup, data$cus))

existing_nodes = read.csv("Data_Dict/Nodes_Data.csv")
existing_nodes = as.character(existing_nodes$ticker)

new_ticker = setdiff(all_companies_ticker, existing_nodes)

access_com_info = data.frame("ticker" = new_ticker)
access_com_info = access_com_info %>% filter(ticker!= 0 &
                                             ticker!= "#N/A N/A",
                                             ticker!= "#N/A Invalid Security",
                                             ticker!= "#N/A Requesting Data...")

get_Name="=IF(ISBLANK(A2),\"\",BDP(A2, \"LONG_COMP_NAME\",\"\"))"
get_country="=IF(ISBLANK(A2),\"\",BDP(A2, \"CNTRY_OF_DOMICILE\",\"\"))"
get_gics="=IF(ISBLANK(A2),\"\",BDP(A2, \"GICS_INDUSTRY_GROUP_NAME\",\"\"))"


access_com_info$Name = NA
access_com_info$country = NA
access_com_info$gics = NA
for(i in 1:(nrow(access_com_info))){
  company = paste0("A", i+1)
  access_com_info$Name[i] = gsub("A2",company, get_Name)
  access_com_info$country[i] = gsub("A2",company, get_country)
  access_com_info$gics[i] = gsub("A2",company, get_gics)
}

# write.csv(access_com_info, "Request/Node_info.csv", row.names = F)
write.xlsx(access_com_info, "Request/Node_info.xlsx", row.names = F)
access_com_info[0:1,]

ticker,Name,country,gics
SILTZ KS Equity,"=IF(ISBLANK(A2),"""",BDP(A2, ""LONG_COMP_NAME"",""""))","=IF(ISBLANK(A2),"""",BDP(A2, ""CNTRY_OF_DOMICILE"",""""))","=IF(ISBLANK(A2),"""",BDP(A2, ""GICS_INDUSTRY_GROUP_NAME"",""""))"


### [Optional] With new Node_Info, update "Nodes_Data.csv"

In [ ]:
update = read.xlsx("Request/Node_info.xlsx", 2)
current = read.csv("Data_Dict/Nodes_Data.csv")
data = rbind(current, update)
data=data[!is.na(data$id) & data$id!="" & data$id!="#N/A Invalid Security",]
#Nodes$id <- str_replace_all(Nodes$id," ","_")
data=data[!duplicated(data),]
write.csv(data, "Data_Dict/Nodes_Data.csv", row.names = F)

### Combine All Nodes Data

# Format and Clean Data

#### The following block is for importing data with different formats

#### For old formats

In [7]:
data = data %>% mutate_all(as.character)

old_format_N = function(data){
    supply_n = data[c(3,4,9,10)]
    cus_n = data[c(5,6,11,12)]
    colnames(supply_n)=c("ticker","id","country","type_label")
    colnames(cus_n)=c("ticker","id","country","type_label")
    Nodes = rbind(supply_n, cus_n)
    remove(supply_n, cus_n)
    return(Nodes)
}

old_format_E = function(data){
    supply = data[c(2,4,7)]
    cus = data[c(6,2,8)]
    colnames(supply)=c("to","from","weight")
    colnames(cus)=c("to","from","weight")
    Edges = rbind(supply, cus)
    remove(cus, supply)
    return(Edges)
}

In [8]:
Nodes = old_format_N(data)
Edges = old_format_E(data)

Nodes[0:1,]
Edges[0:1,]

ticker,id,country,type_label
FLEX US Equity,Flex Ltd,US,Technology Hardware & Equipmen


to,from,weight
Applied Materials Inc,Flex Ltd,74.485176


In [11]:
Edges=Edges[!is.na(Edges$from) & Edges$from!="" & Edges$from!="#N/A Invalid Security",]
Edges=Edges[!is.na(Edges$to) & Edges$to!="" & Edges$to!="#N/A Invalid Security",]
#Edges$from <- str_replace_all(Edges$from," ","_")
#Edges$to <- str_replace_all(Edges$to," ","_")
Edges=Edges[!duplicated(Edges),]

Nodes=Nodes[!is.na(Nodes$id) & Nodes$id!="" & Nodes$id!="#N/A Invalid Security",]
#Nodes$id <- str_replace_all(Nodes$id," ","_")
Nodes=Nodes[!duplicated(Nodes),]

#check complete
for(com in Edges$from){
  if(!(com %in% Nodes$id)){
    print(com)
  }
}
for(com in Edges$to){
  if(!(com %in% Nodes$id)){
    print(com)
  }
}

In [10]:
add = data.frame("ticker" = NA, "id"="SK Siltron Co Ltd", 
                 "country" = "KR", "type_label" = "Semiconductors & Semiconductor")
Nodes = rbind(Nodes, add)

In [14]:
Nodes_Data$country[Nodes_Data$country=="HK"]="CN"

In [12]:
Nodes_Data = Nodes
Edges_Data = Edges
Nodes_Data[0:1,]
Edges_Data[0:1,]

ticker,id,country,type_label
FLEX US Equity,Flex Ltd,US,Technology Hardware & Equipmen


to,from,weight
Applied Materials Inc,Flex Ltd,74.485176


In [13]:
path = getwd()
setwd(paste0(path, "/Temp"))
saveRDS(Nodes_Data, file = "Nodes_Data.rds")
saveRDS(Edges_Data, file = "Edges_Data.rds")